In [3]:
!pip install pycaret
from pycaret.classification import *
!pip install catboost
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')

     |████████████████████████████████| 256kB 5.5MB/s 
     |████████████████████████████████| 6.8MB 14.3MB/s 
     |████████████████████████████████| 14.2MB 50.9MB/s 
     |████████████████████████████████| 245kB 52.4MB/s 
     |████████████████████████████████| 276kB 45.3MB/s 
     |████████████████████████████████| 1.8MB 46.7MB/s 
     |████████████████████████████████| 65.8MB 81kB/s 
     |████████████████████████████████| 102kB 10.4MB/s 
     |████████████████████████████████| 174kB 47.0MB/s 
     |████████████████████████████████| 1.6MB 43.2MB/s 
     |████████████████████████████████| 157.5MB 34kB/s 
     |████████████████████████████████| 61kB 7.4MB/s 
     |████████████████████████████████| 1.1MB 40.4MB/s 
     |████████████████████████████████| 163kB 48.9MB/s 
     |████████████████████████████████| 348kB 37.3MB/s 
     |████████████████████████████████| 81kB 9.3MB/s 
     |████████████████████████████████| 153kB 42.4MB/s 
     |████████████████████████████████| 604kB 36.4MB/

In [6]:
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
from datetime import date, timedelta
import time
from google.colab import drive
drive.mount('/content/drive')
import random
import matplotlib.pyplot as plt
from scipy.stats.mstats import winsorize
from sklearn.cluster import KMeans
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score

Mounted at /content/drive


In [ ]:
train_err = pd.read_csv('/content/drive/Shareddrives/dacon/404/train_err2.csv')
train_qui = pd.read_csv('/content/drive/Shareddrives/dacon/404/train_qui2.csv')
train_prob = pd.read_csv('/content/drive/Shareddrives/dacon/404/train_prob2.csv')

test_err = pd.read_csv('/content/drive/Shareddrives/dacon/404/test_err2.csv')
test_qui = pd.read_csv('/content/drive/Shareddrives/dacon/404/test_qui2.csv')

In [ ]:
path = '/content/drive/Shareddrives/dacon/404/preprocessed/mk2_with_categorized/'

test_err_42_counts = pd.read_csv(path + '테스트_에러_42항목별_카운트.csv', index_col=['Unnamed: 0'])
train_err_42_counts = pd.read_csv(path + '트레인_에러_42항목별_카운트.csv', index_col=['Unnamed: 0'])

test_err_sum_byday = pd.read_csv(path +'테스트_에러발생_일일누적합계.csv', index_col=['Unnamed: 0'])
train_err_sum_byday = pd.read_csv(path +'트레인_에러발생_일일누적합계.csv', index_col=['Unnamed: 0'])

test_qui_sum_byday = pd.read_csv(path +'테스트_퀼리티로그_일일수집합계.csv', index_col=['Unnamed: 0'])
train_qui_sum_byday = pd.read_csv(path +'트레인_퀼리티로그_일일수집합계.csv', index_col=['Unnamed: 0'])

test_2h = pd.read_csv(path +'테스트_err_2시간간격.csv', index_col=['Unnamed: 0'])
train_2h = pd.read_csv(path +'트레인_err_2시간간격.csv', index_col=['Unnamed: 0'])

train_6h = pd.read_csv(path + '트레인_err_6시간간격.csv', index_col=['Unnamed: 0'])
test_6h = pd.read_csv(path + '테스트_err_6시간간격.csv', index_col=['Unnamed: 0'])

train_12h = pd.read_csv(path + '트레인_err_12시간간격.csv', index_col=['Unnamed: 0'])
test_12h = pd.read_csv(path + '테스트_err_12시간간격.csv', index_col=['Unnamed: 0'])

## 퀼리티 데이터를 확률로 환산하여 평균한 값
train_qui_prob = pd.read_csv(path + '트레인_퀼리티_확률.csv')#, index_col=['Unnamed: 0'])
test_qui_prob = pd.read_csv(path + '테스트_퀼리티_확률.csv')#, index_col=['Unnamed: 0'])

train_qui_prob.set_index(keys = 'user_id', inplace=True)
test_qui_prob.set_index(keys = 'user_id', inplace=True)

train_errcode = pd.read_csv(path + '트레인_에러코드_900이상.csv')
test_errcode = pd.read_csv(path + '테스트_에러코드_900이상.csv')

train_errcode.set_index(keys = 'user_id', inplace = True)
test_errcode.set_index(keys = 'user_id', inplace = True)

In [ ]:
### 43262 없따...

In [ ]:
test_errcode

In [ ]:
train_time_series_6h = pd.read_csv(path + 'train_time_series_6h.csv', index_col=['Unnamed: 0'])
test_time_series_6h = pd.read_csv(path + 'test_time_series_6h.csv', index_col=['Unnamed: 0'])
train_time_series_6h.drop(['Group', 'problems'], axis = 1, inplace = True)

prob_list = list(set(train_prob.user_id))

In [ ]:
train_2h.rename(columns = lambda x: x + '_2', inplace = True)
test_2h.rename(columns = lambda x: x + '_2', inplace = True)

train_6h.rename(columns = lambda x: x + '_6', inplace = True)
test_6h.rename(columns = lambda x: x + '_6', inplace = True)

train_12h.rename(columns = lambda x: x + '_12', inplace = True)
test_12h.rename(columns = lambda x: x + '_12', inplace = True)

In [ ]:
# 각 사용자가 42가지 에러타입을 몇 번씩 겪었는지 나타냄
train_err_42_counts

In [ ]:
# 일단 자료들 날짜형식이나 콤마(,) 제거 등 간단한 전처리 진행

train_qui2 = train_qui.copy()
test_qui2 = test_qui.copy()
train_err2 = train_err.copy()
test_err2 = test_err.copy()

# 퀼리티 3,4는 값이 1밖에 없어서 의미가 없기 때문에 제외
# 펌웨어 버전은 이미 err 자료에 있기 때문에 중복으로 제외
# 퀼리티 0은 EDA 결과 퀼리티 2와 피어슨 상관계수 0.99를 보이기 때문에 0과 2 중에 하나만 골랐음
# 0의 결측치 Nan은 15%고 2의 결측치는 4%기 때문에 결측치가 적은 2를 남기고 0은 drop 시킨다.

## 퀼리티 6과 7도 0.81로 강한 상관관계를 보이고 있는데, 하나를 제외해야 하나 고민이 됨
train_qui2.drop(['fwver','quality_0', 'quality_3', 'quality_4'], axis =1, inplace=True)
test_qui2.drop(['fwver', 'quality_0', 'quality_3', 'quality_4'], axis =1, inplace=True)

test_qui2 = test_qui2.replace(',','', regex=True)
train_qui2 = train_qui2.replace(',','', regex=True)
test_qui2.loc[:,'quality_1':'quality_12'] = test_qui2.loc[:,'quality_1':'quality_12'].astype(float)
train_qui2.loc[:,'quality_1':'quality_12'] = train_qui2.loc[:,'quality_1':'quality_12'].astype(float)

train_qui2['time'] = pd.to_datetime(train_qui['time'], format = '%Y-%m-%d %H:%M:%S')
test_qui2['time'] = pd.to_datetime(test_qui['time'], format = '%Y-%m-%d %H:%M:%S')

train_err2['time'] = pd.to_datetime(train_err2['time'], format = '%Y-%m-%d %H:%M:%S')
test_err2['time'] = pd.to_datetime(test_err2['time'], format = '%Y-%m-%d %H:%M:%S')

train_err4 = train_err2.copy()
test_err4 = test_err2.copy()

In [ ]:
##### 퀼리티 데이터, 하루에 2개 이상 수집되는 경우(에러 빈도가 높은 경우)

def quality_one_day(dataset):
  data = dataset.copy()
  ids = data['user_id'].unique()
  ids.sort()
  data['day'] = data['time'].dt.dayofyear
  df_list = []
  for id in tqdm(ids):
    temp = data[data['user_id'] == id]
    d_list = temp.day.unique()
    too_much = 0
    for d in d_list:
      temp2 = temp[temp['day'] == d]
      err = len(temp2)/12
      if err > 1:
        too_much += err
    df_list.append(too_much)

  df = pd.DataFrame(df_list, index = ids, columns = ['overlap'])
  return df

In [ ]:
train_tm = quality_one_day(train_qui2)
test_tm = quality_one_day(test_qui2)

In [ ]:
# 퀼리티 임계치
# [-1, 0, 1] == -1 카테고리 -1, -1~0 =카테고리 0, 그 이상 = 카테고리 1
qualities = {}

qualities['quality_1'] = [-1,0, 1]
qualities['quality_2'] = [-1,0, 1]
qualities['quality_5'] = [-1,0,1,2,4, 10]
qualities['quality_6'] = [-1,0, 1]
qualities['quality_7'] = [0,1,100, 101]
qualities['quality_8'] = [0,1,2]
qualities['quality_9'] = [0,1,2]
qualities['quality_10'] = [0,1,2,3, 4, 5,7, 10 ,14,15] ##??
qualities['quality_11'] = [-1,0,1]
qualities['quality_12'] = [0,1,2]

## 모델 임계치
## 모델 0,1,2,3,4의 비율을 더하면 99.999% 이므로, 모델 5~8은 other model로 통합하는 것이 바람직 해 보임.

In [ ]:
def quality_categorize(dataset):
  data = dataset.copy()
  for key in tqdm(qualities):
    intervals = qualities.get(key)
    for i in range(len(intervals)):
      if intervals[i] == intervals[-1]:
        data[key][data[key] >= intervals[i]] =  intervals[i]
      # elif intervals[i+1] - intervals[i] == 1:
      #   continue
      else:
        data[key][(data[key] >= intervals[i]) & (data[key] < intervals[i+1])] = intervals[i]
  return data

###### 그냥 쓰자니 불일치가 너무 많아서... 카테고리화 해서 확률로 전환한다.

# train_qui3 = quality_categorize(train_qui2)
# test_qui3 = quality_categorize(test_qui2)

In [ ]:
### 펌웨어의 확률을 구하는 함수 ###
## 유진's 코드를 참조하여 좀더 빠르게 구동되도록 수정함.

def fw_probs(traindata, testdata):
  
  train_temp = traindata.drop_duplicates(subset = 'user_id')
  test_temp = testdata.drop_duplicates(subset = 'user_id')
  

  test_list = []
  train_temp['problems'] = train_temp['user_id'].isin(prob_list)
  train_tr = train_temp[train_temp['problems']]
  train_fls = train_temp[train_temp['problems'] != True]

  new_fw = ['03.11', '04.16', '04.22', '04.33', '04.73', '04.82', '05.15', '05.66', '8.5', '10']

  for fw in tqdm(new_fw):
    if fw == '10':
      ori = len(train_temp[(train_temp['fwver'].str.contains(fw)) & ~(train_temp['fwver'].str.contains('104')) & ~(train_temp['fwver'].str.contains('109'))])
      tr = len(train_tr[(train_tr['fwver'].str.contains(fw)) & ~(train_tr['fwver'].str.contains('104')) & ~(train_tr['fwver'].str.contains('109'))])
      fls = len(train_fls[(train_temp['fwver'].str.contains(fw)) & ~(train_fls['fwver'].str.contains('104')) & ~(train_fls['fwver'].str.contains('109'))])
      fw_st = [ori, tr, fls, tr/ori]
      test_list.append(fw_st)
    else:
      ori = len(train_temp[train_temp['fwver'].str.contains(fw)])
      tr = len(train_tr[train_tr['fwver'].str.contains(fw)])
      fls = len(train_fls[train_fls['fwver'].str.contains(fw)])
      fw_st = [ori, tr, fls, tr/ori]
      test_list.append(fw_st)

  fw_df = pd.DataFrame(test_list, index = new_fw, columns = ['원본', '불만고객', '고객', '불만확률'])

  train_temp['fw_prob'] = 0
  test_temp['fw_prob'] = 0

  for fw in tqdm(new_fw):
    if fw == '10':
      train_temp['fw_prob'][(train_temp['fwver'].str.contains(fw)) & ~(train_temp['fwver'].str.contains('104')) & ~(train_temp['fwver'].str.contains('109'))] = fw_df.loc[fw][3]
      test_temp['fw_prob'][(test_temp['fwver'].str.contains(fw)) & ~(test_temp['fwver'].str.contains('104')) & ~(test_temp['fwver'].str.contains('109'))] = fw_df.loc[fw][3]
    else:
      train_temp['fw_prob'][train_temp['fwver'].str.contains(fw)] = fw_df.loc[fw][3]
      test_temp['fw_prob'][test_temp['fwver'].str.contains(fw)] = fw_df.loc[fw][3]
  df = train_temp.loc[:,['user_id', 'fw_prob']]
  df_t = test_temp.loc[:,['user_id', 'fw_prob']]

  df.set_index('user_id', inplace = True)
  df_t.set_index('user_id', inplace = True)

  return df, df_t

In [ ]:
train_fw, test_fw = fw_probs(train_err2, test_err2)

In [ ]:
# Error 데이터를 일일 단위로 쪼개서 각 사용자가 날짜별로 겪은 에러코드의 누계
# 0일은 10월 31일, 마지막은 11월 30일
train_err_sum_byday

In [ ]:
# Quality 데이터를 일일 단위로 쪼개서 퀼리티 로그가 사용자 별로 몇 번 수집되었나
# 누적, 마지막 11월 30일 자료는 최종 몇 번의 로그 수집이 있었는가를 의미함.
train_qui_sum_byday

In [ ]:
# train_err_42_counts, train_err_code_frq, train_qui_frq, train_err_sum_byday, train_qui_sum_byday
# test_err3, test_err_42_counts, test_err_code_frq, test_qui_frq, test_err_sum_byday, test_qui_sum_byday
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
robust = RobustScaler()
robust_q = RobustScaler()
robust_nq = RobustScaler()
minmax = MinMaxScaler()

In [ ]:
# 24시간을 단순하게 평균내면, 23시와 1시의 평균이 이상해지기 때문에(실제로는 2시간 차이인데, 산술적으로 22시간 차이로 잡힘)
# 태양광 때 처럼 일단 코사인 시간으로 바꿔서 평균, 분산, 최빈값을 구한다.

def cos_time(dataset):
  dataset['cos_time'] = np.cos(2*np.pi*dataset.hour/24)
  dataset['sin_time'] = np.sin(2*np.pi*dataset.hour/24)
  return dataset

def cos_stats(dataset):
  dataset['hour'] = dataset['time'].dt.hour
  data = cos_time(dataset)[['user_id', 'cos_time', 'sin_time']]
  cols = []
  temp = data.groupby(by = 'user_id')

  a = temp['cos_time'].mean()
  cols += ['cos_mean']
  b = temp['cos_time'].std()
  cols += ['cos_std']

  c = temp['sin_time'].mean()
  cols += ['sin_mean']
  d = temp['sin_time'].std()
  cols += ['sin_std']

  df = pd.concat([a,b,c,d], axis = 1) 
  df.columns = cols
  return df

In [ ]:
cos_train = cos_stats(train_err4)
cos_test = cos_stats(test_err4)

In [ ]:
cos_test

In [ ]:
# errtype 1~42까지 확률로 치환하기.

def errtype_to_prob(trainset, testset):
  data = trainset.copy()
  t_data = testset.copy()
  data['problems'] = data['user_id'].isin(prob_list)
  one_hot = pd.get_dummies(data['errtype'])

  temp = data['problems']
  temp2 = pd.concat([temp, one_hot], axis = 1)

  temp2_tr = temp2[temp2['problems']].fillna(0)
  temp2_fls = temp2[temp2['problems'] != True].fillna(0)

  trues = pd.DataFrame(temp2_tr.sum()).transpose()
  falses = pd.DataFrame(temp2_fls.sum()).transpose()
  tru = trues.drop('problems', axis = 1)
  fls = falses.drop('problems', axis = 1)
  rslt = tru/(tru+fls)
  er_list = list(set(data.errtype))
  er_list.sort()
  for er in tqdm(er_list):
    data.errtype[data['errtype'] == er] =rslt.loc[0,er]
    t_data.errtype[t_data['errtype'] == er] =rslt.loc[0,er]
  return data, t_data

In [ ]:
train, test = errtype_to_prob(train_err2, test_err2)

In [ ]:
train_errtype_mean = train.groupby(by = 'user_id').mean()[['errtype']]
train_errtype_std = train.groupby(by = 'user_id').std()[['errtype']]
train_errtype_prob = pd.concat([train_errtype_mean, train_errtype_std], axis = 1)
train_errtype_prob.columns = ['errtype_mean', 'errtype_std']


test_errtype_mean = test.groupby(by = 'user_id').mean()[['errtype']]
test_errtype_std = test.groupby(by = 'user_id').std()[['errtype']]
test_errtype_prob = pd.concat([test_errtype_mean, test_errtype_std], axis = 1)
test_errtype_prob.columns = ['errtype_mean', 'errtype_std']

In [ ]:
train_errtype_prob

In [ ]:
# 중간에 펌웨어 업데이트를 한 경우에 1, 아니면 0으로 표시하는 함수 

def update(dataset):

  data = dataset.copy()
  ids = list(set(data.user_id))
  ids.sort()
  print('1/3')
  data.replace({'fwver': np.nan}, {'fwver': '00.00.0000'}, inplace = True)
  data.replace({'fwver': '8.5.3'}, {'fwver': '08.05.3000'}, inplace = True)
  data.replace({'fwver': '10'}, {'fwver': '10.00.0000'}, inplace = True)
  data['fwver'] = data['fwver'].str.replace('\.','', regex = True)
  print('2/3')
  data = data.astype({'fwver': 'int'})
  fw_list = data.fwver.unique()
  data2 = data.groupby(by = 'user_id').mean()
  print('3/3')
  data2['fw_update'] = data2['fwver'].isin(fw_list)
  data2['fw_update'] = data2['fw_update'].astype(int)
  df = data2.loc[:,['fw_update']]
  return df

In [ ]:
fw_update_train = update(train_err2)
fw_update_test = update(test_err2)

In [ ]:
train_err3 = train_err2[['user_id', 'errtype', 'errcode', 'time']]
train_err3['prob'] = train_err3['user_id'].isin(prob_list)
ec = pd.DataFrame(train_err3.errcode.value_counts())
err_code = list(ec[ec['errcode'] > 900].index)

In [ ]:
train_err3 = train_err2[['user_id', 'errtype', 'errcode', 'time']]
test_err3 = test_err2[['user_id', 'errtype', 'errcode', 'time']]

# 상위 15개 리스트
# err_code =['1','0','connection timeout','B-A8002','80','79','14','active','2','84','85','standby','NFANDROID2','connection fail to establish','3']

train_err3['prob'] = train_err3['user_id'].isin(prob_list)
ec = pd.DataFrame(train_err3.errcode.value_counts())
err_code = list(ec[ec['errcode'] > 1000].index)

# 위의 15개에 포함되지 않는 에러코드는 걍 'etc'로 분류
train_err3['errcode'][~train_err3['errcode'].isin(err_code)] = 'ETC'
test_err3['errcode'][~test_err3['errcode'].isin(err_code)] = 'ETC'

In [ ]:
dummi = pd.get_dummies(train_err3['errcode'], prefix = 'Err_code_')
id = train_err3[['user_id']]
train_types = pd.concat([id, dummi], axis =1)

dummi = pd.get_dummies(test_err3['errcode'], prefix = 'Err_code_')
id = test_err3[['user_id']]
test_types = pd.concat([id, dummi], axis =1)

In [ ]:
k = train_types[train_types['user_id'] == 10000]
a = k.groupby(by = 'user_id').sum()

k = train_types[train_types['user_id'] == 10001]
b = k.groupby(by = 'user_id').sum()

In [ ]:
df = pd.DataFrame(columns = train_types.columns)

In [ ]:
df = pd.concat([df,b], axis = 0)
df

In [ ]:
df = pd.concat([df,a], axis = 0)
df

In [ ]:
# 시계열 자료를 활용해 군집화.
ks = KMeans(n_clusters = 7, random_state = 42)
ks.fit(train_time_series_6h)
train_time_series_6h['Group'] = ks.predict(train_time_series_6h)
test_time_series_6h['Group'] = ks.predict(test_time_series_6h)
# train_time_series_6h['problems'] = train_time_series_6h.index.isin(prob_list)

In [ ]:
train_time_series_6h

In [ ]:
def multi_model(dataset):
  data = dataset.copy()
  #ids = [x for x in range(10000,12000)]
  ids = data.user_id.unique()
  ids.sort()

  models = []

  for id in tqdm(ids):
    temp = data[data['user_id'] == id]
    mul = len(temp.model_nm.unique())
    if mul == 1:
      models.append('one')
    else:
      models.append('multi')
    
  df = pd.DataFrame(models, index = ids, columns = ['multi_model'])

  return df

# tt = train_err2[train_err2.user_id == 10001]
# len(tt.fwver.unique())

In [ ]:
train_multi_model = multi_model(train_err2)
test_multi_model = multi_model(test_err2)

In [ ]:
n_path = 'C:/404/404/preprocessed/mk3_dacon/'

a = pd.read_csv(n_path +'a.csv') #, index_col=['Unnamed: 0'])
a.index = a.index + 10000
i = pd.read_csv(n_path +'i.csv') #, index_col=['Unnamed: 0'])
i.index = i.index + 10000
j = pd.read_csv(n_path +'j.csv') #, index_col=['Unnamed: 0'])
j.index = j.index + 10000
l = pd.read_csv(n_path +'l.csv') #, index_col=['Unnamed: 0'])
l.index = l.index + 10000
o = pd.read_csv(n_path +'o.csv') #, index_col=['Unnamed: 0'])
o.index = o.index + 10000
k = pd.read_csv(n_path +'k.csv') #, index_col=['Unnamed: 0'])
k.index = k.index + 10000
g = pd.read_csv(n_path +'g.csv') #, index_col=['Unnamed: 0'])
g.index = g.index + 10000
h = pd.read_csv(n_path +'h.csv') #, index_col=['Unnamed: 0'])
h.index = h.index + 10000

train_bonus = pd.concat([a,g,h,i,j,k,l,o], axis = 1)

test_path = 'C:/404/404/preprocessed/mk3_dacon/test/'

a = pd.read_csv(test_path +'a.csv') #, index_col=['Unnamed: 0'])
a.index = a.index + 30000
i = pd.read_csv(test_path +'i.csv') #, index_col=['Unnamed: 0'])
i.index = i.index + 30000
j = pd.read_csv(test_path +'j.csv') #, index_col=['Unnamed: 0'])
j.index = j.index + 30000
l = pd.read_csv(test_path +'l.csv') #, index_col=['Unnamed: 0'])
l.index = l.index + 30000
o = pd.read_csv(test_path +'o.csv') #, index_col=['Unnamed: 0'])
o.index = o.index + 30000
k = pd.read_csv(test_path +'k.csv') #, index_col=['Unnamed: 0'])
k.index = k.index + 30000
g = pd.read_csv(test_path +'g.csv') #, index_col=['Unnamed: 0'])
g.index = g.index + 30000
h = pd.read_csv(test_path +'h.csv') #, index_col=['Unnamed: 0'])
h.index = h.index + 30000

test_bonus = pd.concat([a,g,h,i,j,k,l,o], axis = 1)

In [ ]:
train_qui0 = pd.read_csv(n_path +'트레인_퀼리티_0말고.csv' , index_col=['Unnamed: 0'])
test_qui0 = pd.read_csv(n_path +'테스트_퀼리티_0말고.csv' , index_col=['Unnamed: 0'])

train_qui_std = pd.read_csv(n_path +'트레인_퀼리티_분산.csv' , index_col=['user_id'])
test_qui_std = pd.read_csv(n_path +'테스트_퀼리티_분산.csv' , index_col=['user_id'])

train_qui_day = pd.read_csv(n_path +'트레인_퀼리티_일일.csv'  , index_col=['Unnamed: 0'])
test_qui_day = pd.read_csv(n_path +'테스트_퀼리티_일일.csv'  , index_col=['Unnamed: 0'])

train_qui_week = pd.read_csv(n_path +'트레인_퀼리티_주간.csv'  , index_col=['Unnamed: 0'])
test_qui_week = pd.read_csv(n_path +'테스트_퀼리티_주간.csv'  , index_col=['Unnamed: 0'])

train_qui_mean = pd.read_csv(n_path +'트레인_퀼리티_평균.csv'  , index_col=['user_id'])
test_qui_mean = pd.read_csv(n_path +'테스트_퀼리티_평균.csv'  , index_col=['user_id'])

train_qui_sum = pd.read_csv(n_path +'트레인_퀼리티_합.csv'  , index_col=['user_id'])
test_qui_sum = pd.read_csv(n_path +'테스트_퀼리티_합.csv'  , index_col=['user_id'])

test_qui_sum.drop(['quality_1_nan'], axis = 1, inplace = True)

In [ ]:
trainset = pd.concat([train_qui0, train_qui_week, train_qui_sum, train_fw, train_bonus, train_err_42_counts, train_err_sum_byday, train_qui_prob, cos_train, train_errtype_prob, fw_update_train, train_errcode,train_tm , train_2h], axis = 1)
testset = pd.concat([test_qui0, test_qui_week, test_qui_sum, test_fw, test_bonus, test_err_42_counts, test_err_sum_byday, test_qui_prob, cos_test, test_errtype_prob, fw_update_test, test_errcode, test_tm, test_2h], axis = 1)


trainset2 = pd.DataFrame(robust.fit_transform(trainset), index = trainset.index, columns=trainset.columns)
testset2 = pd.DataFrame(robust.transform(testset), index = testset.index, columns=testset.columns)
trainset2.fillna(-3, inplace=True)
testset2.fillna(-3, inplace=True)


col = [str(x) + '_t' for x in range(len(trainset2.columns))]
trainset2.columns = col
testset2.columns = col

trainset3 = pd.concat([trainset2, train_time_series_6h, train_multi_model], axis = 1)
testset3 = pd.concat([testset2, test_time_series_6h, test_multi_model], axis = 1)

du = pd.get_dummies(trainset3['Group'], prefix = 'Group')
du2 = pd.get_dummies(trainset3['multi_model'], prefix = 'model')
trainset3 = pd.concat([trainset3, du, du2], axis = 1)
trainset3.drop(['Group', 'multi_model'], axis = 1, inplace=True)
du = pd.get_dummies(testset3['Group'], prefix = 'Group')
du2 = pd.get_dummies(testset3['multi_model'], prefix = 'model')
testset3 = pd.concat([testset3, du, du2], axis = 1)
testset3.drop(['Group', 'multi_model'], axis = 1, inplace=True)



trainset3.reset_index(inplace=True)
probs = list(set(train_prob.user_id))
trainset3['TARGET'] = trainset3['index'].apply(lambda x: 1 if x in probs else 0)
trainset3.set_index('index', inplace = True)

num = list(trainset3.columns)
num.remove('TARGET')

In [ ]:
testset3

In [ ]:
trainset3

In [ ]:
X_train = trainset3.iloc[:,:-1]
y_train = trainset3.iloc[:,-1]

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform

In [ ]:
param_test ={'min_samples_leaf': sp_randint(6, 100), 
             'min_samples_split': sp_randint(1, 500), 
             'max_depth': sp_randint(4,10),
             'subsample': sp_uniform(loc=0.2, scale=0.8), 
             'max_features': sp_randint(1, 500)}


In [ ]:
clf = GradientBoostingClassifier(random_state=42, n_estimators=30, learning_rate=0.3)
rs = RandomizedSearchCV(estimator=clf, param_distributions=param_test, n_iter=50000, scoring='roc_auc', cv=5, refit=True, random_state=42, verbose=True, n_jobs= -1)
rs.fit(X_train, y_train)

print('Best score reached: {} with params: {} '.format(rs.best_score_, rs.best_params_))

In [ ]:
print('Best score reached: {} with params: {} '.format(rs.best_score_, rs.best_params_))